# PhilReinholdPygrape tutorial 5: Measurement and register operations

This notebook is based on the original examples written by Phil,
they can be found in [this directory](https://github.com/tesla-cat/yarn/tree/master/examplesFromForkedLibraries/PhilReinholdPygrape).
Specifically, it contains the following: 

- `cavity_register.py`

Ruiqi, 28 Jun 2020

In [1]:
import numpy as np
from qutip import *
from qutip.qip.algorithms import qft
from yarn.PhilReinholdPygrape import *
from yarn.qutipHelpers import (
    plotWignersIntermediateStates, 
    plotOccupationsIntermediateStates,
    plotExpectation, cat
)

## parameters

### 1. operation 

In [ ]:
operateOnQubitN = 1
operationIsCoherent = True

operationTypes = ['measurement','register']
operationType = operationTypes[0]

measurementShouldInvert = True
measurementNames = ['projectionX','projectionY','projectionZ']
measurementName = measurementNames[0]

operationName = 'qft'

### 2. system

In [2]:
dimC = 20
dimQ = 2
numQ = 3

### 3. frequency (GHz)

In [ ]:
driveC = driveQ = 1e-3
chi = -2.95e-3
chiPrime = 1e-6
anharmonicityQ = -215e-3
kerr = -5.4e-6

### 4. time (ns)

In [ ]:
relaxationC = 2e7
relaxationQ = 70e3
dephasingQ = 22e3

### 5. optimization

In [ ]:
useLoss = True
maxIter = 120 * 2**numQ
maxAmp = 16

## operators

### 1. projection operators for measurement

In [ ]:
g, e = fock(2,0), fock(2,1)
projectionZ = ket2dm( e )
projectionX = ket2dm( (g +    e).unit() )
projectionY = ket2dm( (g + 1j*e).unit() )

### 2. operators for register

In [ ]:
pauli = {'x':sigmax(), 'y':sigmay(), 'z':sigmaz()}
def rotation(axis,angle):
    return ( 1j*angle/2*pauli[axis] ).expm()

pauliX = sigmax()
pauliZ = sigmaz()
rotationXpiOver2 = rotation('x',np.pi/2)
rotationYpiOver2 = rotation('y',np.pi/2)


## define the grape setup

In [18]:
inits = np.concatenate([
    np.array([tensor(basis(dimQ, 0), basis(dimC, i)).full()[:,0] for i in range(n)]),
    np.array([tensor(basis(dimQ, 1), basis(dimC, i)).full()[:,0] for i in range(n)]),
])

U = np.identity(dimC*dimQ, dtype=np.complex)
U[:n,:n] = final_op[:n,:n]
U[dimC:dimC+n,dimC:dimC+n] = final_op[n:,n:]
U[:n,dimC:dimC+n] = final_op[:n,n:]
U[dimC:dimC+n,:n] = final_op[n:,:n]
finals = []
for i in range(inits.shape[0]):
    finals.append(np.dot(U,inits[i,:]))
finals = np.array(finals)

print(inits)
print(finals)

NameError: name 'final_op' is not defined

In [ ]:
def makeSetup(dimC, dimQ):
    Hdrift, HcontrolList = make_hmt(
        dimC, dimQ,   chi, chiPrime, 
        kerr, anharmonicityQ,   driveQ, driveC
    )

    # Incoherent, don't need cavity drive
    if not operationIsCoherent: 
        HcontrolList = HcontrolList[:2,:,:]

    if operationType == 'measurement':
        identities = projections = flips = [qeye(2) for _ in range(numQ+1)]
        identity = tensor(identities)
        # project
        projections[operateOnQubitN-1] = eval(measurementName)
        projections.reverse() # why?
        project = tensor(projections)    
        # flip
        flips[-1] = sigmax()    
        flips.reverse()
        flip = tensor(flips)
        # operation = flip * <project+> + identity * <project->
        if not measurementShouldInvert:
            operation = flip * project + (identity - project)
        else:
            operation = project + flip * (identity - project)
        
        inits = np.concatenate([
            np.array([qutip.tensor(qutip.basis(nq, 0), qutip.basis(nc, i)).full()[:,0] for i in range(n)]),
            np.array([qutip.tensor(qutip.basis(nq, 1), qutip.basis(nc, i)).full()[:,0] for i in range(n)]),
        ])

        U = np.identity(nc*nq, dtype=np.complex)
        U[:n,:n] = final_op[:n,:n]
        U[nc:nc+n,nc:nc+n] = final_op[n:,n:]
        U[:n,nc:nc+n] = final_op[:n,n:]
        U[nc:nc+n,:n] = final_op[n:,:n]
        finals = []
        for i in range(inits.shape[0]):
            finals.append(np.dot(U,inits[i,:]))
        finals = np.array(finals)

    # Register operations
    else:
        ops = [qutip.identity(2) for _ in range(N_BITS)]
        if opname == 'x':
            ops[BIT_N-1] = qutip.sigmax()
        elif opname == 'x2':
            ops[BIT_N-1] = (1j*np.pi/4*qutip.sigmax()).expm()
        elif opname == 'y2':
            ops[BIT_N-1] = (1j*np.pi/4*qutip.sigmay()).expm()
        elif opname == 'z':
            ops[BIT_N-1] = qutip.sigmaz()
        elif opname == 'qft':
            xs = np.arange(n) / float(n)
            ops = [qutip.Qobj(np.array(
                [1/np.sqrt(n)*np.exp(1j*2*np.pi*i*xs) 
                 for i in range(n)]))]
        ops.reverse()
        op = qutip.tensor(ops).full()

        inits = np.array([qutip.tensor(qutip.basis(nq, 0), qutip.basis(nc, i)).full()[:,0] for i in range(n)])
        finals = np.zeros((n,nq*nc), dtype=np.complex)
        finals[:n,:n] = op

    a, ad, b, bd = make_ops(nc, nq)
    if USE_LOSS:
        loss_vec = np.ones(nc*nq) - ((a.dag()*a) * 0.5/T1cav + (b.dag()*b) * 0.5/T1q).diag()
    else:
        loss_vec = None

    return StateTransferSetup(H0, Hcs, inits, finals, coherent=coherent, loss_vec=loss_vec)
